In [34]:
from PIL import Image
import requests
import torch
import torch.nn.functional as F
from transformers import AutoProcessor, AltCLIPModel
from transformers import AltCLIPModel, AltCLIPProcessor

model = AltCLIPModel.from_pretrained("/home/alex/Alt_clip_Wudao/model/checkpoint-258420").to("cuda")
processor = AltCLIPProcessor.from_pretrained("/home/alex/Alt_clip_Wudao/model/checkpoint-258420")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=[ "一只猫和一只狗"], images=image, return_tensors="pt", padding=True).to("cuda")

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

Some weights of the model checkpoint at /home/alex/Alt_clip_Wudao/model/checkpoint-258420 were not used when initializing AltCLIPModel: ['text_model.transformation_pre.weight', 'text_model.transformation_pre.bias']
- This IS expected if you are initializing AltCLIPModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AltCLIPModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
# 用模型计算 # global age

def input_output( text , image):
    
    inputs = processor(text=[ text], images=image, return_tensors="pt", padding=True).to("cuda")
    outputs = model(**inputs)
    
    return outputs
    
    
# 根据结果的得到余弦值
def get_cosine_similarity(outputs):
    text_embeds = outputs.text_embeds
    image_embeds = outputs.image_embeds
    cos_sim = torch.cosine_similarity(image_embeds,text_embeds).detach().cpu().numpy() # 余弦相似度
    cos_sim = str(cos_sim)
    print("余弦相似度 ："+ cos_sim)
    return cos_sim

In [36]:
out_a = input_output("一只猫和一只老鼠",image)
out_b = input_output("一只狗和一只猫",image)

a = get_cosine_similarity(out_a)
b = get_cosine_similarity(out_b)


余弦相似度 ：[0.2478287]
余弦相似度 ：[0.23274015]


In [38]:
# 把list 写进csv
path_folder ="/share/projset/baaishare/baai-mmdataset/wudaomm-5m"

import os
import pandas as pd

path = 'temp.csv'
# 一次写入一行
df = pd.DataFrame(data=[['1', '98.2789%']])
# 一次写入多行
df = pd.DataFrame(data=[['1', '98.2789%'], ['2', '76.2345%']])
# 解决追加模式写的表头重复问题
if not os.path.exists(path):
    df.to_csv(path, header=['epoch', 'accuracy'], index=False, mode='a')
else:
    df.to_csv(path, header=False, index=False, mode='a')
